In [1]:
from simple_modflow.modflow.mf6.sfr import SFR
from pathlib import Path
import pickle
from simple_modflow.modflow.mf6.voronoiplus import VoronoiGridPlus as Vor
import figs as f
import pandas as pd
import flopy
import geopandas as gpd
from simple_modflow.modflow.mf6.mfsimbase import SimulationBase
import shapely as shp
import numpy as np

In [51]:
vor_path = Path(r'C:\Users\lukem\Python\MODFLOW\LakePointe\new_vor_lakepointe.vor')
with open(vor_path, 'rb') as file:
    vor: Vor = pickle.load(file)
with open(Path(r"C:\Users\lukem\Python\MODFLOW\LakePointe\LakePointe.model"),'rb') as file:
    model = pickle.load(file)

In [52]:
input_path = Path(r'C:\Users\lukem\Python\MODFLOW\LakePointe\inputs')
stream = input_path / r'shp\rivers and streams\jenkins.shp'
sfr = SFR(vor=vor, stream_path=stream, model=model, reverse_stream=True)
array: np.ndarray = sfr.sfr_reach_data
sfr.get_smoothed_reach_elevs()

initing sfr
sorting
getting reach data
(0, 145)
(0, 146)
(0, 147)
(0, 148)
(0, 149)
(0, 150)
(0, 151)
(0, 152)
(0, 153)
(0, 154)
(0, 155)
(0, 156)
(0, 157)
(0, 158)
(0, 159)
(0, 160)
(0, 161)
(0, 162)
(0, 163)
(0, 164)
(0, 165)
(0, 166)
(0, 167)
(0, 168)
(0, 169)
(0, 170)
(0, 171)
(0, 172)
(0, 173)
(0, 174)
(0, 175)
(0, 176)
(0, 177)
(0, 178)
(0, 179)
(0, 180)
(0, 181)
(0, 182)
(0, 183)
(0, 184)
(0, 185)
(0, 186)
(0, 187)
(0, 188)
(0, 189)
(0, 190)
(0, 191)
(0, 192)
(0, 193)
(0, 194)
(0, 195)
(0, 196)
(0, 197)
(0, 198)
(0, 199)
(0, 200)
(0, 201)
(0, 202)
(0, 203)
(0, 212)
(0, 213)
(0, 283)
(0, 284)
(0, 285)
(0, 286)
(0, 287)
(0, 288)
(0, 289)
(0, 290)
(0, 291)
(0, 292)
(0, 293)
(0, 294)
(0, 295)
(0, 296)
(0, 297)
(0, 298)
(0, 299)
(0, 300)
(0, 301)
(0, 302)
(0, 303)
(0, 304)
(0, 305)
(0, 306)
(0, 349)
(0, 350)
(0, 352)
(0, 353)
(0, 354)
(0, 355)
(0, 356)
(0, 357)
(0, 358)
(0, 359)
(0, 410)
(0, 411)
(0, 412)
(0, 414)
(0, 415)
(0, 416)
(0, 418)
(0, 419)
(0, 420)
(0, 433)
(0, 441)
(0, 453

{145: 354.87353515625,
 1125: 354.87353515625,
 349: 354.7913818359375,
 1297: 354.7913818359375,
 283: 354.7913818359375,
 146: 354.7913818359375,
 7751: 354.7913818359375,
 284: 354.7913818359375,
 147: 354.7913818359375,
 7749: 354.7913818359375,
 148: 354.7913818359375,
 149: 354.7913818359375,
 614: 354.7913818359375,
 529: 354.7913818359375,
 669: 354.7913818359375,
 1424: 354.7913818359375,
 150: 354.7913818359375,
 1313: 354.7913818359375,
 1237: 354.7913818359375,
 1380: 354.7913818359375,
 285: 354.7913818359375,
 7232: 354.7913818359375,
 151: 354.7913818359375,
 152: 354.7913818359375,
 7913: 354.7913818359375,
 286: 354.7913818359375,
 1512: 354.7913818359375,
 153: 354.7913818359375,
 1497: 354.7913818359375,
 471: 354.7913818359375,
 1177: 354.7913818359375,
 569: 354.7913818359375,
 1174: 354.7913818359375,
 154: 354.7913818359375,
 1168: 354.7913818359375,
 420: 354.7913818359375,
 155: 354.7913818359375,
 1487: 354.7913818359375,
 156: 354.7913818359375,
 287: 354.791

In [16]:
sfr.sfr_connection_data()

TypeError: 'LineString' object is not subscriptable

In [55]:
cells, cells_df = sfr.get_sorted_cells_along_stream()
stream_cells = vor.gdf_topbtm.loc[cells, :][0].reset_index()
fig = f.Fig()
fig.add_scattergl(x=cells_df.distance_along_stream.to_list(), y=stream_cells[0])
fig.show()

In [46]:
stream_cells

{194: 386.2855224609375,
 6882: 386.0721435546875,
 399: 386.0721435546875,
 1040: 386.0721435546875,
 316: 386.0721435546875,
 586: 385.9346008300781,
 554: 385.9346008300781,
 572: 385.9346008300781,
 472: 385.9346008300781,
 520: 385.9346008300781,
 193: 385.9346008300781,
 889: 385.7011413574219,
 401: 385.52410888671875,
 312: 385.051025390625,
 8229: 385.051025390625,
 7986: 384.87786865234375,
 276: 384.65093994140625,
 7980: 384.37603759765625,
 505: 384.1687927246094,
 7976: 384.0384216308594,
 313: 384.0384216308594,
 7640: 383.5047607421875,
 7345: 383.5047607421875,
 7604: 383.487548828125,
 192: 383.487548828125,
 7599: 383.487548828125,
 450: 383.487548828125,
 7991: 383.1263122558594,
 384: 383.1263122558594,
 7994: 383.11212158203125,
 8242: 382.9365539550781,
 191: 382.8205871582031,
 8247: 382.8205871582031,
 8240: 382.6761474609375,
 8257: 382.6761474609375,
 275: 382.6268615722656,
 8272: 382.32672119140625,
 504: 382.23980712890625,
 8278: 382.23980712890625,
 190:

In [47]:
stream_cells = sfr.get_smoothed_reach_elevs(reverse=True)
d = pd.DataFrame.from_dict(stream_cells, orient='index', columns=[0])
vorb = vor.gdf_topbtm.update(d)
vor.gdf_topbtm[0][138]

354.80716

In [48]:
stream_cell_keys = list(stream_cells.keys())
fig =f.Fig()
fig.add_scattergl(x=list(range(len(stream_cells))), y=list(stream_cells.values()))
fig.show()

In [4]:
def generate_lak_period_data(num_stress_periods, lake_ids, lake_stages=None, rainfall_rates=None, evaporation_rates=None, withdrawals=None):
    """
    Generate period data for the MODFLOW 6 LAK package.

    Parameters:
    num_stress_periods (int): Number of stress periods.
    lake_ids (list of int): List of lake IDs.
    lake_stages (list of float, optional): List of lake stages for each stress period.
    rainfall_rates (list of float, optional): List of rainfall rates for each stress period.
    evaporation_rates (list of float, optional): List of evaporation rates for each stress period.
    withdrawals (list of float, optional): List of withdrawal rates for each stress period.

    Returns:
    dict: Dictionary of period data for the LAK package. Each key is a stress period number, and the value is a list of lists.
    """
    period_data = {}

    for period in range(num_stress_periods):
        period_data[period] = []

        for lake_id in lake_ids:
            lake_number = lake_id  # Assuming lake_id is already 0-based for FloPy

            laksetting = [lake_number]
            if lake_stages is not None:
                laksetting.extend(['stage', lake_stages[period]])
            if rainfall_rates is not None:
                laksetting.extend(['rainfall', rainfall_rates[period]])
            if evaporation_rates is not None:
                laksetting.extend(['evaporation', evaporation_rates[period]])
            if withdrawals is not None:
                laksetting.extend(['withdrawal', withdrawals[period]])

            period_data[period].append(laksetting)
    
    return period_data

# Example usage
num_stress_periods = 5  # Example number of stress periods
lake_ids = [0, 1]  # Example lake IDs
lake_stages = [100, 101, 102, 103, 104]  # Example lake stages for each stress period
rainfall_rates = [0.01, 0.02, 0.015, 0.01, 0.02]  # Example rainfall rates for each stress period
evaporation_rates = [0.005, 0.004, 0.006, 0.005, 0.004]  # Example evaporation rates for each stress period
withdrawals = [0.002, 0.003, 0.001, 0.002, 0.003]  # Example withdrawal rates for each stress period
import pandas as pd
lak_period_data = generate_lak_period_data(num_stress_periods, lake_ids, lake_stages, rainfall_rates, evaporation_rates, withdrawals)
pd.DataFrame.from_dict(lak_period_data)


,0,1,2,3,4
0,"[0, stage, 100, rainfall, 0.01, evaporation, 0...","[0, stage, 101, rainfall, 0.02, evaporation, 0...","[0, stage, 102, rainfall, 0.015, evaporation, ...","[0, stage, 103, rainfall, 0.01, evaporation, 0...","[0, stage, 104, rainfall, 0.02, evaporation, 0..."
1,"[1, stage, 100, rainfall, 0.01, evaporation, 0...","[1, stage, 101, rainfall, 0.02, evaporation, 0...","[1, stage, 102, rainfall, 0.015, evaporation, ...","[1, stage, 103, rainfall, 0.01, evaporation, 0...","[1, stage, 104, rainfall, 0.02, evaporation, 0..."
